Sur ce nb:

- Parser/Vectorisation/Modèle
- Evaluation du modèle

Reste à perfectionner le modèle et/ou les autres paramètres. Utiliser un Memory Network pour après?

In [6]:
import re
#from __future__ import print_function
import tarfile
import numpy as np


from keras.utils.data_utils import get_file



def tokenize(sent):
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parserBabi(data):
    ret=list()
    story=list()
    
    for phrase in data:
        phrase = phrase.decode('utf-8').strip()
        id_phrase,phrase = int(phrase[0]),phrase[1:]
        
        if id_phrase == 1: #Nouvelle story
            story=list()
            
        if '\t' in phrase: #Si tabulation alors il s'agit de la question ainsi que de la réponse
            q, a, justif = phrase.split('\t')
            q = tokenize(q)
            
            #la conditionelle permet de ne pas inclure les séparateurs
            data_story = [x for x in story if x] #L'ensemble des élements qui nous permettra de raisonner sur le texte
                                             
            ret.append((data_story,q,a)) #Nos données d'apprentissages
            story.append('')
        
        else: 
            #Alors la phrase est tout simplement un des élements de raisonnement et non une question
            story.append(tokenize(phrase))
            
    return ret

def readAndParse(f):
    data = parserBabi(f.readlines())
    return [([substory for substories in story for substory in substories], q , a) for story,q,a in data]



In [7]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget -O ~/.keras/datasets/babi-tasks-v1-2.tar.gz http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n')
    raise

tar = tarfile.open(path)
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
#challenge = 'tasks_1-20_v1-2/en/qa19_path-finding_{}.txt'

train = readAndParse(tar.extractfile(challenge.format('train')))
test = readAndParse(tar.extractfile(challenge.format('test')))
vocab = sorted(reduce(lambda x, y: x | y, (set(story + q + [answer]) for story, q, answer in train + test)))
vocab_size = len(vocab) + 1
word_idx = dict(((w,i+1) for i,w in enumerate(vocab)))

story_max = max((len(x) for x,_,_ in train+test))
question_max = max((len(x) for _,x,_ in train+test))


In [10]:
#Mettons tout ça dans des matrices
from keras.preprocessing.sequence import pad_sequences

def vectorize(data,word_idx,story_max,question_max):
    
    X = []
    Xq = []
    Y = []

    for story, question, reponse in train:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in question]
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[reponse]] = 1 
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    
    return pad_sequences(X, maxlen=story_max), pad_sequences(Xq, maxlen=question_max), np.array(Y)

X,Xq,Y = vectorize(train,word_idx,story_max,question_max)
X_test,Xq_test,Y_test = vectorize(test,word_idx,story_max,question_max)

In [11]:
print X[0:10]
print Xq[0:10]
print Y[0:10]

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  7 18 21 20 11  1  6 23 21 20 14  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  7 18 21 20 11  1  6 23 21
  20 14  1  5 23 10 21 20 14  1  8 18 21 20 13  1]
 [ 0  0  0  7 18 21 20 11  1  6 23 21 20 14  1  5 23 10 21 20 14  1  8 18
  21 20 13  1  6 18 21 20 19  1  8 16 21 20 11  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  8 22 21 20 19  1  8 23 21 20 11  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8 22 21 20 19  1  8 23
  21 20 11  1  7 23 21 20 12  1  5 18 21 20 14  1]
 [ 0  0  0  0  8 22 21 20 19  1  8 23 21 20 11  1  7 23 21 20 12  1  5 18
  21 20 14  1  6 23 21 20 13  1  6 22 21 20 19  1]


Notes:

- Avec un batchsize de 32 un nombre de 50 epochs semble optimal.
- Le fait d'avoir utilisé un batchsize petit a amelioré les résultats (32 optimal?)


In [41]:
#Model

from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout,RepeatVector,Activation
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop,Adam

embed_size = 50
batch_size=32
epochs=80

story_model = Sequential()
story_model.add(Embedding(vocab_size,embed_size,input_length=story_max))
story_model.add(Dropout(0.3)) #La couche Dropout permet d'éviter sur-apprentissage

question_model = Sequential()
question_model.add(Embedding(vocab_size,embed_size,input_length=question_max))
question_model.add(Dropout(0.3))
question_model.add(LSTM(embed_size))
question_model.add(RepeatVector(story_max)) #permet d'ajuster la taille du modèle afin de préparer un merge

model = Sequential()
model.add(Merge([story_model, question_model], mode='sum'))
model.add(LSTM(embed_size))
model.add(Dropout(0.3))
model.add(Dense(vocab_size))
model.add(Activation("softmax"))

model.compile(optimizer=RMSprop(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit([X, Xq], Y, batch_size=batch_size, nb_epoch=epochs)


Epoch 1/80
1000/1000 [==============================] - 5s - loss: 2.0472 - acc: 0.1630     

In [42]:
loss,acc = model.evaluate([X_test, Xq_test],Y_test, batch_size=batch_size)
print "\nPerte = {}".format(loss)
print "Précision = {}".format(acc)

def getClass(nd_matrix):
    return np.argmax(nd_matrix,axis=1)
    

 992/1000 [============================>.] - ETA: 0s